# ИСПОЛЬЗОВАНИЕ BERT(LaBSE) ДЛЯ КЛАССИФИКАЦИИ ТЕКСТА НА РУССКОМ ЯЗЫКЕ
Данильченко Вадим

In [1]:
# !pip install tensorflow_addons tensorflow_text

импортируем необходимые библиотеки

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_addons.optimizers import AdamW
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_text as text

сделаем тест наличия GPU

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

1. ПРЕПРОЦЕССИНГ И СОЗДАНИЕ ДАТАСЕТА

In [6]:
# импортируем данные
data = pd.read_excel(r'data_train_cleaned short.xlsx', engine='openpyxl')
data.head()

,message,target,label,rnbr
0,жду ответа,1,label_1,1
1,заранее спасибо,1,label_1,2
2,спасибо заранее,1,label_1,3
3,lf,10,label_10,1
4,ok,10,label_10,2


In [7]:
# уменьшим объем для быстрого эксперимента
data = data[data['rnbr']<=20]
data.shape

(5102, 4)

In [8]:
# приведем сообщения к типу str
data['message'] = data['message'].astype('str')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5102 entries, 0 to 77611
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  5102 non-null   object
 1   target   5102 non-null   int64 
 2   label    5102 non-null   object
 3   rnbr     5102 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 199.3+ KB


In [9]:
# посмотрим сколько у нас классов
classes = len(data['target'].unique().tolist())
classes

278

In [10]:
# подготовим таргет
x = data['message'].tolist()
y = data['target'].tolist()
y_cat = tf.keras.utils.to_categorical(y)

In [11]:
# разобьем на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x, y_cat, test_size=0.1, random_state=777)

In [12]:
# преобразуем в датасет
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 512
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

2. МОДЕЛЬ

In [13]:
# выгрузим предобученную модель для препроцессинга текста, а также LaBSE на основе BERT
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2", trainable=False)

In [20]:
# архитектура модели - текст из датасета пропускается через препроцессор и LaBSE, выход которого забирается
# и прогоняется через полносвязные слои с дропаутом, на последнем шаге слой сколичеством необхимых нам классов 
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(preprocessor, name='preprocessing')(text_input)
encoder_layer = hub.KerasLayer(encoder, trainable=False, name='LaBSE_encoder')(preprocessing_layer)
x = encoder_layer['pooled_output']
x = tf.keras.layers.Flatten(name='flatten')(x)
x = tf.keras.layers.Dropout(0.1, name='dropout1')(x)
x = tf.keras.layers.Dense(768, activation=tf.keras.activations.relu, name='dense1')(x)
x = tf.keras.layers.Dropout(0.2, name='dropout2')(x)
x = tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, name='dense2')(x)
x = tf.keras.layers.Dropout(0.2, name='dropout3')(x)
output = tf.keras.layers.Dense(y_train.shape[1], activation=tf.keras.activations.softmax, name='dense_output')(x)

model = tf.keras.Model(text_input, output)

In [21]:
# компилируем модель и выводим архитектуру
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              # optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [22]:
# проведем обучение модели
model.fit(train_ds,
          validation_data=test_ds,
          epochs=20,
          batch_size=batch_size,
          verbose=1)

Epoch 1/20
9/9 [==============================] - 116s 13s/step - loss: 5.4546 - accuracy: 0.0340 - val_loss: 5.0060 - val_accuracy: 0.1879
Epoch 2/20
9/9 [==============================] - 112s 13s/step - loss: 4.3081 - accuracy: 0.2501 - val_loss: 3.3798 - val_accuracy: 0.4990
Epoch 3/20
9/9 [==============================] - 112s 13s/step - loss: 2.7757 - accuracy: 0.4622 - val_loss: 2.0055 - val_accuracy: 0.6419
Epoch 4/20
9/9 [==============================] - 113s 13s/step - loss: 1.7494 - accuracy: 0.6203 - val_loss: 1.3608 - val_accuracy: 0.7280
Epoch 5/20
9/9 [==============================] - 112s 13s/step - loss: 1.1472 - accuracy: 0.7286 - val_loss: 1.0586 - val_accuracy: 0.7476
Epoch 6/20
9/9 [==============================] - 113s 13s/step - loss: 0.8353 - accuracy: 0.7959 - val_loss: 0.8817 - val_accuracy: 0.7789
Epoch 7/20
9/9 [==============================] - 113s 13s/step - loss: 0.6431 - accuracy: 0.8364 - val_loss: 0.7866 - val_accuracy: 0.8082
Epoch 8/20
9/9 [====

In [23]:
# посмотрим на результат
model.evaluate(test_ds)

1/1 [==============================] - 12s 12s/step - loss: 0.6260 - accuracy: 0.8415


[0.6259775757789612, 0.8414872884750366]

да, наблюдается переобучение, но в целом для тестового эксперимента этих показателей достаточно